In [1]:
from torchsummary import summary
from cspresunet import CSPResUNet
from resunet import ResUNet
from unet import UNet
from datasets import EchoNetDataset
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

ModuleNotFoundError: No module named 'cspresunet.cspunet_parts'

In [2]:
model = CSPUNet(1,1)

In [3]:
summary(model, input_size=(1, 112, 112), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 56, 56]             216
       BatchNorm2d-2           [-1, 24, 56, 56]              48
              ReLU-3           [-1, 24, 56, 56]               0
            Conv2d-4           [-1, 24, 56, 56]           5,184
       BatchNorm2d-5           [-1, 24, 56, 56]              48
              ReLU-6           [-1, 24, 56, 56]               0
            Conv2d-7           [-1, 24, 56, 56]             576
       BatchNorm2d-8           [-1, 24, 56, 56]              48
       FusedMBConv-9           [-1, 24, 56, 56]               0
           Conv2d-10           [-1, 24, 56, 56]           5,184
      BatchNorm2d-11           [-1, 24, 56, 56]              48
             ReLU-12           [-1, 24, 56, 56]               0
           Conv2d-13           [-1, 24, 56, 56]             576
      BatchNorm2d-14           [-1, 24,

In [4]:
class LitSeg(pl.LightningModule):
    def __init__(self,
                 backbone,
                 learning_rate=1e-4,
                 root_dir="/home/tienyu/data/EchoNet-Dynamic"):
        super().__init__()
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.root_dir = root_dir

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        inputs, _, masks, _ = batch
        input_frames = torch.cat(inputs[1:])
        masks = torch.cat(masks).float()

        logits = self(input_frames)
        loss = F.binary_cross_entropy(logits, masks)

        tensorboard_logs = {'loss': {'train': loss}}
        self.log('loss', loss, logger=True)
        return {"loss": loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        inputs, _, masks, _ = batch
        input_frames = torch.cat(inputs[1:])
        masks = torch.cat(masks).float()

        logits = self(input_frames)
        loss = F.binary_cross_entropy(logits, masks)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_end(self, outputs):
        val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'loss': {'val': val_loss}}
        return {"val_loss": val_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),
                                      lr=self.learning_rate)
        return optimizer

    # Data hooks
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.echo_train = EchoNetDataset(root_dir=self.root_dir,
                                             split="train")
            self.echo_val = EchoNetDataset(root_dir=self.root_dir, split="val")

        if stage == 'test' or stage is None:
            self.echo_test = EchoNetDataset(root_dir=self.root_dir,
                                            split="test")

    def train_dataloader(self):
        return DataLoader(self.echo_train,
                          batch_size=8,
                          shuffle=True,
                          num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.echo_val, batch_size=8, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.echo_test, batch_size=8, num_workers=4)

In [5]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
backbone = CSPUNet(1, 1)
model = LitSeg(backbone=backbone)
tb_logger = pl_loggers.TensorBoardLogger('logs_cspunet/')
model_checkpoint = ModelCheckpoint(monitor='val_loss', every_n_val_epochs=1)
trainer = pl.Trainer(gpus=AVAIL_GPUS,
                     max_epochs=30,
                     progress_bar_refresh_rate=20,
                     logger=tb_logger,
                     callbacks=[model_checkpoint])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [6]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type    | Params
-------------------------------------
0 | backbone | CSPUNet | 3.6 M 
-------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.267    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/tienyu/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:895: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn('Detected KeyboardInterrupt, attempting graceful shutdown...')


In [7]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 0.27056142687797546}
--------------------------------------------------------------------------------


[{'val_loss': 0.27056142687797546}]

In [9]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir logs_cspunet/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
